In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from libary import *
from recurrent import *
from ARMA import *

In [2]:
train_ratio = 0.5
valid_ratio = 0.25

# Select frequency
frequencies = ['min','5min','15min','60min','D']
frequencies_number_of_samples = np.multiply([390,78,26,7,1],252)

In [3]:
for frequency_index in range(4, -1, -1):
    print(f'Frequency: {frequencies[frequency_index]}')
    number_of_study_periods, study_periods, Data, dates = creating_study_periods(frequencies,\
                                                                                 frequencies_number_of_samples,\
                                                                                 frequency_index)
    # visualize_data(Data, Returns)
    
    print('ARMA')
    ARMA_parameters, ARMA_mse, ARMA_predictions = train_ARMA(number_of_study_periods, study_periods,\
                                                             train_ratio, valid_ratio,\
                                                             frequency_index, frequencies, frequencies_number_of_samples)
    
    print('LSTM')
    LSTM_names, LSTM_mse, LSTM_predictions = train_recurrent_model('LSTM', number_of_study_periods ,study_periods,\
                                                             train_ratio, valid_ratio,\
                                                             frequency_index, frequencies, frequencies_number_of_samples)
    print('GRU')
    GRU_names, GRU_mse, GRU_predictions = train_recurrent_model('GRU', number_of_study_periods, study_periods,\
                                                             train_ratio, valid_ratio,\
                                                             frequency_index, frequencies, frequencies_number_of_samples)
    
    
    
    visualize_results((np.concatenate((np.reshape(ARMA_mse[:,-1], [number_of_study_periods,1]),\
                          np.reshape(LSTM_mse[:,-1], [number_of_study_periods,1]),\
                          np.reshape(GRU_mse[:,-1], [number_of_study_periods,1]),\
                          np.reshape(np.mean(np.square(study_periods[0,:,-ARMA_predictions.shape[1]:]),axis=1),\
                                                         [number_of_study_periods,1])), axis=1)))

Frequency: D
ARMA
Period: 0, order: [0. 0.], mse: [4.61537205e-04 9.17540686e-05]
Period: 1, order: [1. 0.], mse: [1.69159138e-04 6.97803921e-05]
Period: 2, order: [0. 0.], mse: [8.54570920e-05 2.47004967e-04]
Period: 3, order: [1. 0.], mse: [1.21962137e-04 5.64653156e-05]
Period: 4, order: [0. 0.], mse: [5.47238238e-05 2.93307003e-05]
Period: 5, order: [0. 0.], mse: [3.65947036e-05 5.76563209e-05]
Period: 6, order: [1. 0.], mse: [4.28818112e-05 1.25444774e-04]
Period: 7, order: [0. 0.], mse: [7.62030212e-05 2.42941148e-05]
ARMA training time: 0.08 minutes
LSTM
Period: 0
|   iter    |  target   | batch_... |  dropout  | first_... | layer_... | learni... | look_back | n_layers  | optimizer |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -0.000305 |  7.031    |  0.4221   |  4.944    |  0.6071   |  1.986    |  28.26    |  2.191    |  0.2566   |
|  2        | -0.000308 |  8.055    |  0.2799   |  30.08

KeyboardInterrupt: 

In [ ]:
model_names = ['ARMA', 'LSTM', 'GRU']

for frequency_index in range(4, -1, -1):
    print(f'Frequency: {frequencies[frequency_index]}')
    number_of_study_periods, study_periods, Data, dates = creating_study_periods(frequencies,\
                                                                                 frequencies_number_of_samples,\
                                                                                 frequency_index)
    study_periods_direction = study_periods[0]
    study_periods_direction[0<study_periods_direction] = 1
    study_periods_direction[0==study_periods_direction] = 0
    study_periods_direction[0>study_periods_direction] = -1
    
    train_size = np.round(np.size(study_periods[0], 1) * train_ratio).astype(int)
    valid_size = np.round(np.size(study_periods[0], 1) * valid_ratio).astype(int)
    test_size = (np.size(study_periods[0], 1) - train_size - valid_size).astype(int)
    
    predictions = np.zeros((3,number_of_study_periods,test_size))
    for model_index in range(3):
        print(model_names[model_index])
        predictions[model_index] = pd.read_csv('results/'+str(model_names[model_index])+'_predictions_frequency_'\
                                         +str(frequencies[frequency_index])+'.csv', header=None).dropna(axis='columns').values
print(predictions)

In [ ]:
import statsmodels.api as sm
i = 7
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(study_periods[0,i], lags=40, ax=ax1) # 
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(study_periods[0,i], lags=40, ax=ax2)# , lags=40